In [1]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
import chime
from sklearn.metrics import roc_auc_score
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingGridSearchCV

In [2]:
%load_ext chime

# Halving search on an XGBoost classifier
Kaggle Playground Series - Season 3, Episode 7

https://www.kaggle.com/competitions/playground-series-s3e7/overvie



# Data preparation

In [3]:
train = pd.read_csv('./data/train.csv', index_col='id'); train

,no_of_adults,no_of_children,no_of_weekend_nights,no_of_week_nights,type_of_meal_plan,required_car_parking_space,room_type_reserved,lead_time,arrival_year,arrival_month,arrival_date,market_segment_type,repeated_guest,no_of_previous_cancellations,no_of_previous_bookings_not_canceled,avg_price_per_room,no_of_special_requests,booking_status
id,,,,,,,,,,,,,,,,,,
0,2,0,0,2,1,0,0,9,2018,1,14,1,1,11,0,67.50,0,0
1,2,0,1,2,0,0,0,117,2018,7,29,0,0,0,0,72.25,0,0
2,2,0,0,1,0,0,0,315,2018,12,2,0,0,0,0,52.00,0,0
3,1,0,0,2,1,0,0,32,2018,12,1,1,0,0,0,56.00,0,0
4,2,0,1,0,0,0,0,258,2018,10,16,0,0,0,0,100.00,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42095,3,0,0,4,0,0,1,160,2018,12,30,1,0,0,0,140.00,2,1
42096,2,0,0,3,0,0,0,34,2017,9,23,0,0,0,0,224.67,0,0
42097,2,0,0,2,2,0,0,292,2018,7,21,0,0,0,0,96.00,0,0


In [4]:
X_train = train.drop(columns='booking_status')
y_train = train.booking_status

In [5]:
scaler = StandardScaler().fit(X_train)
X_train = pd.DataFrame(scaler.transform(X_train), 
                       columns=X_train.columns, 
                       index=X_train.index)
X_train

,no_of_adults,no_of_children,no_of_weekend_nights,no_of_week_nights,type_of_meal_plan,required_car_parking_space,room_type_reserved,lead_time,arrival_year,arrival_month,arrival_date,market_segment_type,repeated_guest,no_of_previous_cancellations,no_of_previous_bookings_not_canceled,avg_price_per_room,no_of_special_requests
id,,,,,,,,,,,,,,,,,
0,0.151040,-0.313454,-0.998814,-0.278849,1.294624,-0.160945,-0.515173,-1.170469,0.409661,-2.330398,-0.214091,0.428551,5.766758,33.699120,-0.101479,-0.998052,-0.737691
1,0.151040,-0.313454,0.130259,-0.278849,-0.407020,-0.160945,-0.515173,0.161740,0.409661,-0.209778,1.473487,-1.149928,-0.173408,-0.060506,-0.101479,-0.870153,-0.737691
2,0.151040,-0.313454,-0.998814,-0.979466,-0.407020,-0.160945,-0.515173,2.604122,0.409661,1.557404,-1.564154,-1.149928,-0.173408,-0.060506,-0.101479,-1.415406,-0.737691
3,-1.753925,-0.313454,-0.998814,-0.278849,1.294624,-0.160945,-0.515173,-0.886758,0.409661,1.557404,-1.676659,0.428551,-0.173408,-0.060506,-0.101479,-1.307702,-0.737691
4,0.151040,-0.313454,0.130259,-1.680084,-0.407020,-0.160945,-0.515173,1.901012,0.409661,0.850531,0.010919,-1.149928,-0.173408,-0.060506,-0.101479,-0.122955,-0.737691
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42095,2.056005,-0.313454,-0.998814,1.122386,-0.407020,-0.160945,0.685890,0.692156,0.409661,1.557404,1.585992,0.428551,-0.173408,-0.060506,-0.101479,0.954088,1.842849
42096,0.151040,-0.313454,-0.998814,0.421768,-0.407020,-0.160945,-0.515173,-0.862087,-2.441040,0.497095,0.798456,-1.149928,-0.173408,-0.060506,-0.101479,3.233919,-0.737691
42097,0.151040,-0.313454,-0.998814,-0.278849,2.996268,-0.160945,-0.515173,2.320411,0.409661,-0.209778,0.573445,-1.149928,-0.173408,-0.060506,-0.101479,-0.230659,-0.737691


### XGBoost

In [6]:
from xgboost import XGBClassifier

In [14]:
params = {
    'n_estimators': [10, 25, 50, 75, 100, 150, 250, 500, 750, 1000],
    'max_depth': [2, 3, 4],
    'learning_rate': [0.05, 0.1, 0.2, 0.3, 0.5, 1],
    'reg_alpha': [1e-4, 5e-4, 7e-4, 0.001, 0.005, 0.007, 0.01, 0.02],
    'reg_lambda': [0.05, 0.07, 0.1, 0.15, 0.2, 0.3, 0.5]
}

In [15]:
xgboost_clf = XGBClassifier(random_state=8,
                         max_leaves=0,
                         n_jobs=-1)

In [16]:
search = HalvingGridSearchCV(
    estimator=xgboost_clf,
    param_grid=params,
    factor=2,
    resource='n_samples',
    max_resources='auto',
    min_resources=2631,
    aggressive_elimination=True,
    cv=5,
    scoring='roc_auc',
    refit=True, 
    return_train_score=False,
    random_state=8, 
    n_jobs=-1,
    verbose=1
)

In [17]:
%%time
%%chime
search.fit(X_train, y_train)

n_iterations: 14
n_required_iterations: 14
n_possible_iterations: 5
min_resources_: 2631
max_resources_: 42100
aggressive_elimination: True
factor: 2
----------
iter: 0
n_candidates: 10080
n_resources: 2631
Fitting 5 folds for each of 10080 candidates, totalling 50400 fits
----------
iter: 1
n_candidates: 5040
n_resources: 2631
Fitting 5 folds for each of 5040 candidates, totalling 25200 fits
----------
iter: 2
n_candidates: 2520
n_resources: 2631
Fitting 5 folds for each of 2520 candidates, totalling 12600 fits
----------
iter: 3
n_candidates: 1260
n_resources: 2631
Fitting 5 folds for each of 1260 candidates, totalling 6300 fits
----------
iter: 4
n_candidates: 630
n_resources: 2631
Fitting 5 folds for each of 630 candidates, totalling 3150 fits
----------
iter: 5
n_candidates: 315
n_resources: 2631
Fitting 5 folds for each of 315 candidates, totalling 1575 fits
----------
iter: 6
n_candidates: 158
n_resources: 2631
Fitting 5 folds for each of 158 candidates, totalling 790 fits
-----

In [18]:
search.best_params_

{'learning_rate': 0.3,
 'max_depth': 2,
 'n_estimators': 100,
 'reg_alpha': 0.0007,
 'reg_lambda': 0.1}

In [19]:
search.best_score_

0.8833828159380168

In [20]:
pd.Series(search.cv_results_['mean_test_score']).describe()

count    20162.000000
mean         0.863673
std          0.011587
min          0.808465
25%          0.859945
50%          0.868337
75%          0.871824
max          0.884929
dtype: float64

### The best score is 1.5% lower than the best score for manually selected hyperparameters (0.8983).
Let's find where the best manually selected HP set was rejected.

In [33]:
results = pd.DataFrame(search.cv_results_)
results

,iter,n_resources,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_learning_rate,param_max_depth,param_n_estimators,param_reg_alpha,param_reg_lambda,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0,2631,0.030887,0.007563,0.004798,0.001336,0.05,2,10,0.0001,0.05,"{'learning_rate': 0.05, 'max_depth': 2, 'n_est...",0.815370,0.825570,0.751587,0.832311,0.817533,0.808474,0.029073,20107
1,0,2631,0.017958,0.000704,0.004590,0.000771,0.05,2,10,0.0001,0.07,"{'learning_rate': 0.05, 'max_depth': 2, 'n_est...",0.815370,0.825570,0.751587,0.832311,0.817533,0.808474,0.029073,20107
2,0,2631,0.016478,0.001006,0.003934,0.000409,0.05,2,10,0.0001,0.1,"{'learning_rate': 0.05, 'max_depth': 2, 'n_est...",0.815370,0.825570,0.751587,0.832311,0.817533,0.808474,0.029073,20107
3,0,2631,0.018727,0.002115,0.008571,0.007893,0.05,2,10,0.0001,0.15,"{'learning_rate': 0.05, 'max_depth': 2, 'n_est...",0.815370,0.825570,0.751587,0.832311,0.817533,0.808474,0.029073,20107
4,0,2631,0.017503,0.001943,0.005417,0.003617,0.05,2,10,0.0001,0.2,"{'learning_rate': 0.05, 'max_depth': 2, 'n_est...",0.815370,0.825570,0.751587,0.832311,0.817533,0.808474,0.029073,20107
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20157,12,21048,0.584729,0.013415,0.008491,0.001101,0.3,2,100,0.0007,0.1,"{'learning_rate': 0.3, 'max_depth': 2, 'n_esti...",0.883084,0.890288,0.875593,0.890189,0.873773,0.882585,0.006985,9
20158,12,21048,0.591708,0.024782,0.008308,0.000246,0.3,2,100,0.0005,0.1,"{'learning_rate': 0.3, 'max_depth': 2, 'n_esti...",0.883084,0.890288,0.875594,0.890189,0.873773,0.882586,0.006985,8
20159,12,21048,0.854028,0.049737,0.009077,0.000497,0.2,2,150,0.005,0.05,"{'learning_rate': 0.2, 'max_depth': 2, 'n_esti...",0.881669,0.889210,0.875211,0.889362,0.874007,0.881892,0.006576,14
20160,13,42096,1.179015,0.038615,0.014271,0.003246,0.3,2,100,0.0007,0.1,"{'learning_rate': 0.3, 'max_depth': 2, 'n_esti...",0.876783,0.888132,0.882590,0.889728,0.879681,0.883383,0.004913,6


Best HPs selected manually:

In [35]:
best_params_manual = \
{'learning_rate': 0.3,
 'max_depth': 3,
 'n_estimators': 700,
 'reg_alpha': 0.0005,
 'reg_lambda': 0.2}

The closest HP set used in the halving search only differs in the number of estimators, which is 750, while the number chosen manually was 700.

In [38]:
closest = results \
    .drop(columns=['mean_fit_time', 'std_fit_time', 'mean_score_time', 'std_score_time']) \
    .query('param_learning_rate == 0.3 & \
           param_max_depth == 3 &\
           param_n_estimators == 750 &\
           param_reg_alpha == 0.0005 &\
           param_reg_lambda == 0.2')
closest

,iter,n_resources,param_learning_rate,param_max_depth,param_n_estimators,param_reg_alpha,param_reg_lambda,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
6059,0,2631,0.3,3,750,0.0005,0.2,"{'learning_rate': 0.3, 'max_depth': 3, 'n_esti...",0.833333,0.861257,0.841071,0.866249,0.846618,0.849706,0.012323,17359


It was rejected at the first iteration with a rather poor performnace of 0.8497!

Checking the same HP set on the full train set:

In [39]:
from sklearn.model_selection import cross_val_score

In [41]:
scores = cross_val_score(xgboost_clf,
                         X_train, y_train,
                         scoring='roc_auc',
                         cv=5,
                         n_jobs=-1)

In [42]:
scores.mean()

0.8958364288942657

This is close to the manually selected set with n_estimators = 700.

# Conclusion
The min_resources parameter should not be too small, otherwise the best performing HP set can be left out at an early iteration!

### 2nd round
Regular grid search for fine tuning. Using the results of the halving search for the purity of experiment

In [25]:
from sklearn.model_selection import GridSearchCV

In [26]:
params_fine = {
    'n_estimators': [80, 90, 100, 110, 120, 130, 140],
    'max_depth': [2, 3],
    'learning_rate': [0.25, 0.3, 0.4],
    'reg_alpha': [6e-4, 7e-4, 8e-4, 9e-4],
    'reg_lambda': [0.08, 0.09, 0.1, 0.11, 0.12, 0.13, 0.14]
}

In [27]:
xgboost_clf = XGBClassifier(random_state=8,
                         max_leaves=0,
                         n_jobs=-1)

In [28]:
model = GridSearchCV(estimator=xgboost_clf,
                     param_grid=params_fine,
                     scoring='roc_auc',
                     cv = 5,
                    verbose = 3)

In [29]:
%%time
%%chime
model.fit(X_train, y_train)

Fitting 5 folds for each of 1176 candidates, totalling 5880 fits
[CV 1/5] END learning_rate=0.25, max_depth=2, n_estimators=80, reg_alpha=0.0006, reg_lambda=0.08;, score=0.872 total time=   0.3s
[CV 2/5] END learning_rate=0.25, max_depth=2, n_estimators=80, reg_alpha=0.0006, reg_lambda=0.08;, score=0.882 total time=   0.3s
[CV 3/5] END learning_rate=0.25, max_depth=2, n_estimators=80, reg_alpha=0.0006, reg_lambda=0.08;, score=0.878 total time=   0.3s
[CV 4/5] END learning_rate=0.25, max_depth=2, n_estimators=80, reg_alpha=0.0006, reg_lambda=0.08;, score=0.886 total time=   0.3s
[CV 5/5] END learning_rate=0.25, max_depth=2, n_estimators=80, reg_alpha=0.0006, reg_lambda=0.08;, score=0.875 total time=   0.3s
[CV 1/5] END learning_rate=0.25, max_depth=2, n_estimators=80, reg_alpha=0.0006, reg_lambda=0.09;, score=0.872 total time=   0.3s
[CV 2/5] END learning_rate=0.25, max_depth=2, n_estimators=80, reg_alpha=0.0006, reg_lambda=0.09;, score=0.882 total time=   0.3s
[CV 3/5] END learning_rat

[CV 4/5] END learning_rate=0.25, max_depth=2, n_estimators=80, reg_alpha=0.0007, reg_lambda=0.13;, score=0.886 total time=   0.3s
[CV 5/5] END learning_rate=0.25, max_depth=2, n_estimators=80, reg_alpha=0.0007, reg_lambda=0.13;, score=0.875 total time=   0.3s
[CV 1/5] END learning_rate=0.25, max_depth=2, n_estimators=80, reg_alpha=0.0007, reg_lambda=0.14;, score=0.872 total time=   0.3s
[CV 2/5] END learning_rate=0.25, max_depth=2, n_estimators=80, reg_alpha=0.0007, reg_lambda=0.14;, score=0.882 total time=   0.3s
[CV 3/5] END learning_rate=0.25, max_depth=2, n_estimators=80, reg_alpha=0.0007, reg_lambda=0.14;, score=0.878 total time=   0.3s
[CV 4/5] END learning_rate=0.25, max_depth=2, n_estimators=80, reg_alpha=0.0007, reg_lambda=0.14;, score=0.886 total time=   0.3s
[CV 5/5] END learning_rate=0.25, max_depth=2, n_estimators=80, reg_alpha=0.0007, reg_lambda=0.14;, score=0.875 total time=   0.3s
[CV 1/5] END learning_rate=0.25, max_depth=2, n_estimators=80, reg_alpha=0.0008, reg_lambd

[CV 3/5] END learning_rate=0.25, max_depth=2, n_estimators=80, reg_alpha=0.0009, reg_lambda=0.12;, score=0.878 total time=   0.3s
[CV 4/5] END learning_rate=0.25, max_depth=2, n_estimators=80, reg_alpha=0.0009, reg_lambda=0.12;, score=0.886 total time=   0.4s
[CV 5/5] END learning_rate=0.25, max_depth=2, n_estimators=80, reg_alpha=0.0009, reg_lambda=0.12;, score=0.875 total time=   0.4s
[CV 1/5] END learning_rate=0.25, max_depth=2, n_estimators=80, reg_alpha=0.0009, reg_lambda=0.13;, score=0.872 total time=   0.3s
[CV 2/5] END learning_rate=0.25, max_depth=2, n_estimators=80, reg_alpha=0.0009, reg_lambda=0.13;, score=0.882 total time=   0.3s
[CV 3/5] END learning_rate=0.25, max_depth=2, n_estimators=80, reg_alpha=0.0009, reg_lambda=0.13;, score=0.878 total time=   0.3s
[CV 4/5] END learning_rate=0.25, max_depth=2, n_estimators=80, reg_alpha=0.0009, reg_lambda=0.13;, score=0.886 total time=   0.3s
[CV 5/5] END learning_rate=0.25, max_depth=2, n_estimators=80, reg_alpha=0.0009, reg_lambd

[CV 2/5] END learning_rate=0.25, max_depth=2, n_estimators=90, reg_alpha=0.0007, reg_lambda=0.11;, score=0.884 total time=   0.3s
[CV 3/5] END learning_rate=0.25, max_depth=2, n_estimators=90, reg_alpha=0.0007, reg_lambda=0.11;, score=0.879 total time=   0.4s
[CV 4/5] END learning_rate=0.25, max_depth=2, n_estimators=90, reg_alpha=0.0007, reg_lambda=0.11;, score=0.887 total time=   0.3s
[CV 5/5] END learning_rate=0.25, max_depth=2, n_estimators=90, reg_alpha=0.0007, reg_lambda=0.11;, score=0.876 total time=   0.3s
[CV 1/5] END learning_rate=0.25, max_depth=2, n_estimators=90, reg_alpha=0.0007, reg_lambda=0.12;, score=0.873 total time=   0.4s
[CV 2/5] END learning_rate=0.25, max_depth=2, n_estimators=90, reg_alpha=0.0007, reg_lambda=0.12;, score=0.884 total time=   0.3s
[CV 3/5] END learning_rate=0.25, max_depth=2, n_estimators=90, reg_alpha=0.0007, reg_lambda=0.12;, score=0.879 total time=   0.4s
[CV 4/5] END learning_rate=0.25, max_depth=2, n_estimators=90, reg_alpha=0.0007, reg_lambd

[CV 1/5] END learning_rate=0.25, max_depth=2, n_estimators=90, reg_alpha=0.0009, reg_lambda=0.1;, score=0.873 total time=   0.4s
[CV 2/5] END learning_rate=0.25, max_depth=2, n_estimators=90, reg_alpha=0.0009, reg_lambda=0.1;, score=0.884 total time=   0.4s
[CV 3/5] END learning_rate=0.25, max_depth=2, n_estimators=90, reg_alpha=0.0009, reg_lambda=0.1;, score=0.879 total time=   0.4s
[CV 4/5] END learning_rate=0.25, max_depth=2, n_estimators=90, reg_alpha=0.0009, reg_lambda=0.1;, score=0.887 total time=   0.4s
[CV 5/5] END learning_rate=0.25, max_depth=2, n_estimators=90, reg_alpha=0.0009, reg_lambda=0.1;, score=0.876 total time=   0.4s
[CV 1/5] END learning_rate=0.25, max_depth=2, n_estimators=90, reg_alpha=0.0009, reg_lambda=0.11;, score=0.873 total time=   0.4s
[CV 2/5] END learning_rate=0.25, max_depth=2, n_estimators=90, reg_alpha=0.0009, reg_lambda=0.11;, score=0.884 total time=   0.4s
[CV 3/5] END learning_rate=0.25, max_depth=2, n_estimators=90, reg_alpha=0.0009, reg_lambda=0.1

...

[CV 4/5] END learning_rate=0.4, max_depth=3, n_estimators=130, reg_alpha=0.0006, reg_lambda=0.09;, score=0.899 total time=   0.7s
[CV 5/5] END learning_rate=0.4, max_depth=3, n_estimators=130, reg_alpha=0.0006, reg_lambda=0.09;, score=0.891 total time=   0.7s
[CV 1/5] END learning_rate=0.4, max_depth=3, n_estimators=130, reg_alpha=0.0006, reg_lambda=0.1;, score=0.888 total time=   0.7s
[CV 2/5] END learning_rate=0.4, max_depth=3, n_estimators=130, reg_alpha=0.0006, reg_lambda=0.1;, score=0.899 total time=   0.7s
[CV 3/5] END learning_rate=0.4, max_depth=3, n_estimators=130, reg_alpha=0.0006, reg_lambda=0.1;, score=0.894 total time=   0.7s
[CV 4/5] END learning_rate=0.4, max_depth=3, n_estimators=130, reg_alpha=0.0006, reg_lambda=0.1;, score=0.899 total time=   0.7s
[CV 5/5] END learning_rate=0.4, max_depth=3, n_estimators=130, reg_alpha=0.0006, reg_lambda=0.1;, score=0.891 total time=   0.7s
[CV 1/5] END learning_rate=0.4, max_depth=3, n_estimators=130, reg_alpha=0.0006, reg_lambda=0.1

[CV 3/5] END learning_rate=0.4, max_depth=3, n_estimators=130, reg_alpha=0.0008, reg_lambda=0.08;, score=0.894 total time=   0.7s
[CV 4/5] END learning_rate=0.4, max_depth=3, n_estimators=130, reg_alpha=0.0008, reg_lambda=0.08;, score=0.900 total time=   0.7s
[CV 5/5] END learning_rate=0.4, max_depth=3, n_estimators=130, reg_alpha=0.0008, reg_lambda=0.08;, score=0.891 total time=   0.7s
[CV 1/5] END learning_rate=0.4, max_depth=3, n_estimators=130, reg_alpha=0.0008, reg_lambda=0.09;, score=0.888 total time=   0.7s
[CV 2/5] END learning_rate=0.4, max_depth=3, n_estimators=130, reg_alpha=0.0008, reg_lambda=0.09;, score=0.899 total time=   0.7s
[CV 3/5] END learning_rate=0.4, max_depth=3, n_estimators=130, reg_alpha=0.0008, reg_lambda=0.09;, score=0.894 total time=   0.7s
[CV 4/5] END learning_rate=0.4, max_depth=3, n_estimators=130, reg_alpha=0.0008, reg_lambda=0.09;, score=0.899 total time=   0.7s
[CV 5/5] END learning_rate=0.4, max_depth=3, n_estimators=130, reg_alpha=0.0008, reg_lambd

[CV 2/5] END learning_rate=0.4, max_depth=3, n_estimators=130, reg_alpha=0.0009, reg_lambda=0.14;, score=0.898 total time=   0.7s
[CV 3/5] END learning_rate=0.4, max_depth=3, n_estimators=130, reg_alpha=0.0009, reg_lambda=0.14;, score=0.895 total time=   0.7s
[CV 4/5] END learning_rate=0.4, max_depth=3, n_estimators=130, reg_alpha=0.0009, reg_lambda=0.14;, score=0.899 total time=   0.7s
[CV 5/5] END learning_rate=0.4, max_depth=3, n_estimators=130, reg_alpha=0.0009, reg_lambda=0.14;, score=0.891 total time=   0.7s
[CV 1/5] END learning_rate=0.4, max_depth=3, n_estimators=140, reg_alpha=0.0006, reg_lambda=0.08;, score=0.888 total time=   0.7s
[CV 2/5] END learning_rate=0.4, max_depth=3, n_estimators=140, reg_alpha=0.0006, reg_lambda=0.08;, score=0.898 total time=   0.8s
[CV 3/5] END learning_rate=0.4, max_depth=3, n_estimators=140, reg_alpha=0.0006, reg_lambda=0.08;, score=0.894 total time=   0.7s
[CV 4/5] END learning_rate=0.4, max_depth=3, n_estimators=140, reg_alpha=0.0006, reg_lambd

[CV 1/5] END learning_rate=0.4, max_depth=3, n_estimators=140, reg_alpha=0.0007, reg_lambda=0.13;, score=0.888 total time=   0.7s
[CV 2/5] END learning_rate=0.4, max_depth=3, n_estimators=140, reg_alpha=0.0007, reg_lambda=0.13;, score=0.899 total time=   0.8s
[CV 3/5] END learning_rate=0.4, max_depth=3, n_estimators=140, reg_alpha=0.0007, reg_lambda=0.13;, score=0.894 total time=   0.7s
[CV 4/5] END learning_rate=0.4, max_depth=3, n_estimators=140, reg_alpha=0.0007, reg_lambda=0.13;, score=0.900 total time=   0.7s
[CV 5/5] END learning_rate=0.4, max_depth=3, n_estimators=140, reg_alpha=0.0007, reg_lambda=0.13;, score=0.891 total time=   0.8s
[CV 1/5] END learning_rate=0.4, max_depth=3, n_estimators=140, reg_alpha=0.0007, reg_lambda=0.14;, score=0.887 total time=   0.7s
[CV 2/5] END learning_rate=0.4, max_depth=3, n_estimators=140, reg_alpha=0.0007, reg_lambda=0.14;, score=0.898 total time=   0.7s
[CV 3/5] END learning_rate=0.4, max_depth=3, n_estimators=140, reg_alpha=0.0007, reg_lambd

[CV 5/5] END learning_rate=0.4, max_depth=3, n_estimators=140, reg_alpha=0.0009, reg_lambda=0.11;, score=0.892 total time=   0.7s
[CV 1/5] END learning_rate=0.4, max_depth=3, n_estimators=140, reg_alpha=0.0009, reg_lambda=0.12;, score=0.888 total time=   0.7s
[CV 2/5] END learning_rate=0.4, max_depth=3, n_estimators=140, reg_alpha=0.0009, reg_lambda=0.12;, score=0.899 total time=   0.7s
[CV 3/5] END learning_rate=0.4, max_depth=3, n_estimators=140, reg_alpha=0.0009, reg_lambda=0.12;, score=0.894 total time=   0.8s
[CV 4/5] END learning_rate=0.4, max_depth=3, n_estimators=140, reg_alpha=0.0009, reg_lambda=0.12;, score=0.899 total time=   0.7s
[CV 5/5] END learning_rate=0.4, max_depth=3, n_estimators=140, reg_alpha=0.0009, reg_lambda=0.12;, score=0.892 total time=   0.7s
[CV 1/5] END learning_rate=0.4, max_depth=3, n_estimators=140, reg_alpha=0.0009, reg_lambda=0.13;, score=0.888 total time=   1.0s
[CV 2/5] END learning_rate=0.4, max_depth=3, n_estimators=140, reg_alpha=0.0009, reg_lambd

In [44]:
model.best_params_

{'learning_rate': 0.4,
 'max_depth': 3,
 'n_estimators': 140,
 'reg_alpha': 0.0007,
 'reg_lambda': 0.09}

In [45]:
model.best_score_

0.894744178719623

This is closer to the manually selected hyperparameter set, although still a bit lower.

In [43]:
pd.Series(model.cv_results_['mean_test_score']).describe()

count    1176.000000
mean        0.887681
std         0.004327
min         0.878591
25%         0.884104
50%         0.888086
75%         0.891543
max         0.894744
dtype: float64

# Prediction

In [46]:
test = pd.read_csv('./data/test.csv', index_col='id'); test

,no_of_adults,no_of_children,no_of_weekend_nights,no_of_week_nights,type_of_meal_plan,required_car_parking_space,room_type_reserved,lead_time,arrival_year,arrival_month,arrival_date,market_segment_type,repeated_guest,no_of_previous_cancellations,no_of_previous_bookings_not_canceled,avg_price_per_room,no_of_special_requests
id,,,,,,,,,,,,,,,,,
42100,3,0,1,4,0,0,1,111,2018,12,5,1,0,0,0,115.60,2
42101,2,0,0,3,0,0,0,22,2017,10,21,0,0,0,0,85.00,0
42102,2,2,0,1,0,0,3,18,2018,8,10,1,0,0,0,240.00,1
42103,2,0,0,3,0,0,0,88,2018,5,30,0,0,0,0,80.75,0
42104,1,0,0,2,2,0,0,7,2018,9,21,1,0,0,0,144.00,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70163,2,0,1,2,0,0,0,315,2018,9,30,1,0,0,0,160.00,0
70164,2,0,2,2,0,0,0,81,2018,3,25,0,0,0,0,65.00,1
70165,2,0,1,2,0,0,0,40,2018,10,22,0,0,0,0,85.00,0


In [47]:
X_test = pd.DataFrame(scaler.transform(test), columns=test.columns, index=test.index)
X_test

,no_of_adults,no_of_children,no_of_weekend_nights,no_of_week_nights,type_of_meal_plan,required_car_parking_space,room_type_reserved,lead_time,arrival_year,arrival_month,arrival_date,market_segment_type,repeated_guest,no_of_previous_cancellations,no_of_previous_bookings_not_canceled,avg_price_per_room,no_of_special_requests
id,,,,,,,,,,,,,,,,,
42100,2.056005,-0.313454,0.130259,1.122386,-0.407020,-0.160945,0.685890,0.087728,0.409661,1.557404,-1.226638,0.428551,-0.173408,-0.060506,-0.101479,0.297092,1.842849
42101,0.151040,-0.313454,-0.998814,0.421768,-0.407020,-0.160945,-0.515173,-1.010111,-2.441040,0.850531,0.573445,-1.149928,-0.173408,-0.060506,-0.101479,-0.526846,-0.737691
42102,0.151040,4.129777,-0.998814,-0.979466,-0.407020,-0.160945,3.088017,-1.059452,0.409661,0.143658,-0.664112,0.428551,-0.173408,-0.060506,-0.101479,3.646696,0.552579
42103,0.151040,-0.313454,-0.998814,0.421768,-0.407020,-0.160945,-0.515173,-0.195983,0.409661,-0.916652,1.585992,-1.149928,-0.173408,-0.060506,-0.101479,-0.641282,-0.737691
42104,-1.753925,-0.313454,-0.998814,-0.278849,2.996268,-0.160945,-0.515173,-1.195140,0.409661,0.497095,0.573445,0.428551,-0.173408,-0.060506,-0.101479,1.061793,-0.737691
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70163,0.151040,-0.313454,0.130259,-0.278849,-0.407020,-0.160945,-0.515173,2.604122,0.409661,0.497095,1.585992,0.428551,-0.173408,-0.060506,-0.101479,1.492610,-0.737691
70164,0.151040,-0.313454,1.259332,-0.278849,-0.407020,-0.160945,-0.515173,-0.282330,0.409661,-1.623525,1.023466,-1.149928,-0.173408,-0.060506,-0.101479,-1.065367,0.552579
70165,0.151040,-0.313454,0.130259,-0.278849,-0.407020,-0.160945,-0.515173,-0.788076,0.409661,0.850531,0.685950,-1.149928,-0.173408,-0.060506,-0.101479,-0.526846,-0.737691


In [49]:
y_pred = model.predict_proba(X_test)
y_pred

array([[0.87500066, 0.12499934],
       [0.8912785 , 0.10872149],
       [0.5924397 , 0.4075603 ],
       ...,
       [0.91690344, 0.08309656],
       [0.38811135, 0.61188865],
       [0.09545875, 0.90454125]], dtype=float32)

In [50]:
result = pd.DataFrame({'id': X_test.index, 'booking_status':y_pred[:, 1]})
result

,id,booking_status
0,42100,0.124999
1,42101,0.108721
2,42102,0.407560
3,42103,0.075411
4,42104,0.587280
...,...,...
28063,70163,0.947388
28064,70164,0.101949
28065,70165,0.083097
28066,70166,0.611889


In [51]:
result.to_csv('halving_search_xgboost.csv', index=False)

# Conclusion
The min_resources parameter was too small so the best manually seleted HP was left out at the first iteration. The resulting score was a bit lower both on crocc-validation and the public test set at the platform.
Need to retry with a larger min. samples.